Explanations with LRP and IG followed by SpRAy in a NN with multiple inputs on a DNN trained in
a dataset designed by a company which wants to hire data scientists among people
who successfully pass some courses which conduct by the company. Many people signup for their training.
Company wants to know which of these candidates are really wants to work for the company after training or looking for a new
employment. Information related to demographics, education, experience are in hands from
candidates signup and enrollment. The DNN predicts the probability of a candidate will work for the company.

**Features:**

enrollee_id : Unique ID for candidate

city: City code

city_ development _index : Developement index of the city (scaled)

gender: Gender of candidate

relevent_experience: Relevant experience of candidate

enrolled_university: Type of University course enrolled if any

education_level: Education level of candidate

major_discipline :Education major discipline of candidate

experience: Candidate total experience in years

company_size: No of employees in current employer's company

company_type : Type of current employer

lastnewjob: Difference in years between previous job and current job

training_hours: training hours completed

target: 0 – Not looking for job change, 1 – Looking for a job change

Note:

The dataset is imbalanced.

Most features are categorical (Nominal, Ordinal, Binary), some with high cardinality.


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tempfile, sys, os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np

import pandas as pd

import tensorflow as tf

from keras.models import Model
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, ThresholdedReLU

from deepexplain.tensorflow import DeepExplain

from numpy import loadtxt
from numpy.random import seed

from sklearn.neighbors import kneighbors_graph
from sklearn.decomposition import PCA
from keras.models import load_model
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split


%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patheffects as PathEffects
sns.set_style('darkgrid', {'axes.facecolor': '.9'})
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')

from scipy import sparse
from scipy import linalg

import time


RANDOM_SEED = 123
seed(RANDOM_SEED)

ImportError: cannot import name LayerNormalization

Load data

In [9]:
# # load the dataset
# dataset = loadtxt('pima-indians-diabetes.csv', delimiter=',')
# # split into input (X) and output (y) variables
# X = dataset[:,0:8]
# y = dataset[:,8]

import os
import glob
for dirname in glob.glob('./data/input/*'):
    print(dirname)


./data/input/sample_submission.csv
./data/input/aug_train.csv
./data/input/aug_test.csv


In [ ]:
train = pd.read_csv('./data/input/aug_train.csv')

test = pd.read_csv('./data/input/aug_test.csv')

In [15]:
labels = train.columns.get_values().tolist()
labels

['enrollee_id',
 'city',
 'city_development_index',
 'gender',
 'relevent_experience',
 'enrolled_university',
 'education_level',
 'major_discipline',
 'experience',
 'company_size',
 'company_type',
 'last_new_job',
 'training_hours',
 'target']

Clean data

In [16]:
train = train.dropna()
test = test.dropna()

### Exploratory Data Analysis

In [21]:
import pandas_profiling as pp
pp.ProfileReport(train)

/home/andrea/anaconda3/envs/xai/lib/python2.7/site-packages/pandas_profiling/plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/andrea/anaconda3/envs/xai/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/andrea/anaconda3/envs/xai/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/andrea/anaconda3/envs/xai/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/andrea/anaconda3/envs/xai/lib/python2.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/andrea/anaconda3/envs/xai

Number of variables,15
Number of observations,8955
Total Missing (%),0.0%
Total size in memory,1.0 MiB
Average record size in memory,120.0 B
Numeric,4
Categorical,10
Boolean,1
Date,0
Text (Unique),0
Rejected,0


Balance in data

In [18]:
train['target'].value_counts()

0.0    7472
1.0    1483
Name: target, dtype: int64

Let's look at how the number of Data Scientists who change the job varies across features.

In [19]:
print(pd.pivot_table(train, values='target',
                    columns=['relevent_experience'], aggfunc=np.sum).T.sort_values('target', ascending=False))

print(pd.pivot_table(train, values='target',
                    columns=['education_level'], aggfunc=np.sum).T.sort_values('target', ascending=False))

print(pd.pivot_table(train, values='target',
                    columns=['enrolled_university'], aggfunc=np.sum).T.sort_values('target', ascending=False))

print(pd.pivot_table(train, values='target',
                    columns=['gender'], aggfunc=np.sum).T.sort_values('target', ascending=False))

print(pd.pivot_table(train, values='target',
                    columns=['major_discipline'], aggfunc=np.sum).T.sort_values('target', ascending=False))

print(pd.pivot_table(train, values='target',
                    columns=['company_type'], aggfunc=np.sum).T.sort_values('target', ascending=False))

                         target
relevent_experience            
Has relevent experience  1260.0
No relevent experience    223.0
                 target
education_level        
Graduate         1092.0
Masters           357.0
Phd                34.0
                     target
enrolled_university        
no_enrollment        1169.0
Full time course      232.0
Part time course       82.0
        target
gender        
Male    1340.0
Female   131.0
Other     12.0
                  target
major_discipline        
STEM              1379.0
Humanities          40.0
Business Degree     21.0
Other               19.0
No Major            15.0
Arts                 9.0
                     target
company_type               
Pvt Ltd              1133.0
Funded Startup        108.0
Public Sector          94.0
Early Stage Startup    78.0
NGO                    55.0
Other                  15.0


The number of people who change the job vary significantly and inconsistenly.
We will need to encode them manually to improve the model.

In [22]:
from sklearn.preprocessing import LabelEncoder

# I do this manually to explicitly tell the model that a better education & experience serves well as a trustworthy input.

# However, later we wil see the feature importanes report in SHAP and notice interesting results.
experience_dict = {'Has relevent experience' : 1,
             'No relevent experience': 0}

education_dict = {'Graduate' : 2,
             'Masters' : 1,
             'Phd' : 0}

enrollment_dict = {'no_enrollment' : 2,
             'Full time course' : 1,
             'Part time course' : 0}

gender_dict = {'Male' : 2,
             'Female' : 1,
             'Other' : 0}

discipline_dict = {'STEM' : 5,
             'Humanities' : 4,
             'Business Degree' : 3,
             'Other' : 2,
             'No Major' : 1,
             'Arts' : 0 }

company_dict = {'Pvt Ltd' : 5,
             'Funded Startup' : 4,
             'Public Sector' : 3,
             'Early Stage Startup' : 2,
             'NGO' : 1,
             'Other' : 0 }


# Train encoding
le = LabelEncoder()
train['gender'] = train['gender'].map(gender_dict)
train['relevent_experience'] = train['relevent_experience'].map(experience_dict)
train['education_level'] = train['education_level'].map(education_dict)
train['enrolled_university'] = train['enrolled_university'].map(enrollment_dict)
train['major_discipline'] = train['major_discipline'].map(discipline_dict)
train['experience'] = le.fit_transform(train['experience'].astype(str))
train['company_size'] = le.fit_transform(train['company_size'].astype(str))
train['company_type'] = train['company_type'].map(company_dict)
train['last_new_job'] = le.fit_transform(train['last_new_job'].astype(str))
#train['city'] = le.fit_transform(train['city'].astype(str))

train = pd.get_dummies(train, columns=['city']) # I do one-hot encoding here, since a higher value of the encoded feature is not related to the 'importance' of a feature.

# Test encoding
test['gender'] = le.fit_transform(test['gender'].astype(str))
test['relevent_experience'] = test['relevent_experience'].map(experience_dict)
test['education_level'] = test['education_level'].map(education_dict)
test['enrolled_university'] = test['enrolled_university'].map(enrollment_dict)
test['major_discipline'] = test['major_discipline'].map(discipline_dict)
test['experience'] = le.fit_transform(test['experience'].astype(str))
test['company_size'] = le.fit_transform(test['company_size'].astype(str))
test['company_type'] = test['company_type'].map(company_dict)
test['last_new_job'] = le.fit_transform(test['last_new_job'].astype(str))
#test['city'] = le.fit_transform(test['city'].astype(str))


test = pd.get_dummies(test, columns=['city'])

In [23]:
X = train.drop('target', axis=1)
y = train['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)
# Further in this notebook we will use 'val' for validation dataset, since we have all the corresponding data and columns unlike in the 'test' dataset.
# Test dataset does not contain the target and thus we will not be able to measure the performance of the model.

Model building

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
data = []
index = []

attributions = dict()

n_1_class = np.count_nonzero(preds)
n_0_class = len(preds)-n_1_class
n_explanations = X.shape[0]

eX_1_lrp = np.zeros((n_1_class,8))
eX_0_lrp = np.zeros((n_0_class,8))
eX_1_ig = np.zeros((n_1_class,8))
eX_0_ig = np.zeros((n_0_class,8))


method_attribution_dict = {'ig_1': ("ig_attributtions_1.txt", eX_1_ig),
                                      'lrp_1': ('lrp_attributions_1.txt', eX_1_lrp),
                                      'ig_0': ("ig_attributtions_0.txt", eX_0_ig),
                                      'lrp_0': ('lrp_attributions_0.txt', eX_0_lrp)}

In [ ]:
# # Statistics
# times_lrp = np.ndarray((n_explanations,))
# times_ig = np.ndarray((n_explanations,))
#
# # Counters of explanations per class
# j_0 = 0
# j_1 = 0
#
# for i in range(n_explanations):
#
#     # input to explain
#     xs = X[[i]]
#
#     # For dividing explanations for SpRAy
#     x_class = preds[i]
#
#     # Explanations
#     with DeepExplain(session=K.get_session()) as de:
#
#         # Need to reconstruct the graph in DeepExplain context, using the same weights.
#         # With Keras this is very easy:
#         # 1. Get the input tensor to the original model
#         input_tensor = model.layers[0].input
#
#         # 2. We now target the output of the last dense layer (pre-softmax)
#         # To do so, create a new model sharing the same layers untill the last dense (index -2)
#         fModel = Model(inputs=input_tensor, outputs = model.layers[-2].output)
#         target_tensor = fModel(input_tensor)
#
#         # method 1
#
#         # parameters
#         arg = 100
#         method_name = 'IG'
#
#         # explanations
#         begin = time.time()
#         explanation = de.explain('intgrad', target_tensor, input_tensor, xs, steps=arg)
#         time.sleep(1)
#         end = time.time()
#
#         # records
#         attributions[method_name] = explanation
#         times_ig[i] = end - begin
#
#         if x_class == 0:
#             eX_0_ig[j_0] = explanation
#         else:
#             eX_1_ig[j_1] = explanation
#
#
#         # method 2
#
#         # parameters
#         arg = 0.0001
#         method_name = 'LRP'
#
#         # explanation
#         begin = time.time()
#         explanation = de.explain('elrp', target_tensor, input_tensor, xs, epsilon=arg)
#         time.sleep(1)
#         end = time.time()
#
#         # record
#         if x_class == 0:
#             eX_0_lrp[j_0] = explanation
#             j_0 += 1
#         else:
#             eX_1_lrp[j_1] = explanation
#             j_1 += 1
#
#         attributions[method_name] = explanation
#         times_lrp[i] = end - begin

Save explanations and report running time

In [ ]:
# method_name = 'IG'
# method_dict = {"Mean time": times_ig.mean()}
# data.append(method_dict)
# index.append(method_name)
#
# method_name = 'LRP'
# method_dict = {"Mean time": times_lrp.mean()}
# data.append(method_dict)
# index.append(method_name)
#
# time_table = pd.DataFrame(data, index=index)
# print(time_table)
#
# for (filename, mat) in method_attribution_dict.values():
#     mat = np.matrix(mat)
#     with open(filename,'wb') as f:
#         for line in mat:
#             np.savetxt(f, line, fmt='%.2f')
#


Show some examples of explanations

In [ ]:
# labels = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
#            'BMI', 'DiabetesPedigreeFunction', 'Age']
# colors = ["blue", "pink", "red", "yellow", "green", "brown", "teal", "goldenrod"]
#
# fig     = plt.figure(figsize = (15, 15))
# rows    = 1
# columns = len(attributions)
#
# for i, method in enumerate(attributions.keys()):
#
#     ax1 = fig.add_subplot(rows, columns, i+1)
#     ax1.set_title(method)
#     ax1.set_xlabel("Input features")
#     ax1.set_ylabel("Attributions")
#     plt.setp(ax1.get_xticklabels(), rotation=45, ha="right",
#          rotation_mode="anchor")
#     barlist = plt.bar(labels, attributions[method].reshape(-1,), color ='black', width = 0.4)
#     for j in range(len(labels)):
#         barlist[j].set_color(colors[j])
#
# plt.show()

### Spectral relevance analysis (SpRAy)

Explanation techniques enable inspection of the decision process on a single instance
basis. SpRAy offers an alternative to efficiently investigate classifier behavior on large datasets, instead of
screening through many individual explanations.
SpRAy applies (spectral) clustering on a dataset of explanations in order to identify typical, as well as
atypical decision behaviors/prediction strategies of the model, and presents them to the user in a
concise and interpretable manner.

It consists of the next steps:

- Step 1: Computation of the explanations for the samples of interest.

- Step 2: Downsizing/normalizing of the explanations, and make them uniform in shape and size.

- Step 3: Cluster analysis on the explanations.A clustering algorithm is applied to the normalized
  explanation.

  For example, in eigengap analysis, the eigenvalue spectrum of spectral cluster encodes information about the cluster structure of
  the relevance maps. A strong increase in the difference between two successive eigenvalues (eigengap) indicates
  well-separated clusters, including atypical classification strategies. The few detected clusters are then presented to
  the user for inspection.


  **Alternately, the clustering step can be replaced by a low-dimensional embedding step to produce a visual map of the
  overall decision structure of the ML model.**

- Step 4: Identification of interesting clusters. Examples with the same cluster index can be understood as being
 associated with some prototypical decision strategy.


- Step 5 (Optional): Visualization by t-stochastic neighborhood embedding (t-SNE). This last step is not part of the
  analysis strictly speaking, but it's usefull to visualize how SpRAy works.


### SpRAy (with k-means)

### Load explanations

Besides explanations made by LRP and IG, we load explanations of LIME computed in other notebook.

In [ ]:
method_attribution_dict = {'ig_1': ("ig_attributtions_1.txt", 0),
                                      'lrp_1': ('lrp_attributions_1.txt', 0),
                                      'ig_0': ("ig_attributtions_0.txt", 0),
                                      'lrp_0': ('lrp_attributions_0.txt', 0),
                                      'lime_1': ("lime_attributtions_1.txt", 0),
                                      'lime_0': ('lime_attributtions_0.txt', 0)}

In [ ]:
for method, (filename, mat) in method_attribution_dict.items():
    # Read attribution matrix of method
    mat = np.loadtxt(filename, dtype=float)
    print(filename + " dims = ", mat.shape)
    method_attribution_dict[method] = (filename, mat)


Normalize explanations

**Note that this above interpretations require that our input data is normalized. Without that, we can’t claim that
importance is proportional to weights.**

In [ ]:
from sklearn import preprocessing

In [ ]:
for method, (filename, mat) in method_attribution_dict.items():
    # Normalize attributions
    scaler = preprocessing.StandardScaler().fit(mat)
    method_attribution_dict[method] = (filename, scaler.transform(mat))


Selection of number of clusters by elbow criteria.



In [ ]:
method_elbow_dict = dict()
method_elbow_dict['ig_1'] = 2
method_elbow_dict['ig_0'] = 2
method_elbow_dict['lrp_1'] = 2
method_elbow_dict['lrp_0'] = 2
method_elbow_dict['lime_1'] = 2
method_elbow_dict['lime_0'] = 2


fig     = plt.figure(figsize = (15, 15))
rows    = len(method_attribution_dict)
columns = 2
i = 0

for method, (filename, mat) in method_attribution_dict.items():

    # Calcule number of clusters using k-means

    inertias = []

    k_candidates = range(1, 10)

    for k in k_candidates:
        k_means = KMeans(random_state=42, n_clusters=k)
        k_means.fit(mat)
        inertias.append(k_means.inertia_)

    ax = fig.add_subplot(rows, columns, i+1)

    sns.scatterplot(x=k_candidates, y = inertias, s=80, ax=ax)
    e = method_elbow_dict[method] - 1
    sns.scatterplot(x=[k_candidates[e]], y = [inertias[e]], color=sns_c[3], s=150, ax=ax)
    sns.lineplot(x=k_candidates, y = inertias, alpha=0.5, ax=ax)

    ax.set(title='Inertia K-Means '+method, ylabel='inertia', xlabel='k')

    i += 1


### Visualization of explanations (PCA + k-means)

All the scatterplots show PCA projections of explanations onto firts two principal components.
The explanations in a scatterplot correspond only to one method and to one predicted class, e.g.
"ig_1" is the shorthand for explanations of Integrated gradient (IG), classified as members of class 1
by the model.

Scatterplots at left are colored by their true class, and the ones at right are colored by k-means
cluster.

Barplots below are examples of explanations grouped by cluster.


In [ ]:
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})




# Utility function to visualize the outputs with t-SNE or PCA

# In order to visualize the results of clustering algorithms, fashion_scatter()
# function takes two arguments: 1. x, which is a 2-D numpy array containing the output of the
# algorithm and 2. colors, which is 1-D numpy array containing the labels of the dataset. The function
# will render a scatter plot with as many unique colors as the number of classes in the variable colors.

# Utility function to visualize the outputs of PCA and t-SNE

def fashion_scatter(x, colors, method):
    # choose a color palette with seaborn.
    num_classes = len(np.unique(colors))
    palette = np.array(sns.color_palette("hls", num_classes))

    # create a scatter plot.
    f = plt.figure(figsize=(6, 6))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40, c=palette[colors.astype(np.int)])
    plt.title(method)

    # add the labels for each digit corresponding to the label
    txts = []

    for i in range(num_classes):

        # Position of each label at median of data points.
        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

def get_pca_proj(mat):
    """Return PCA projection and prints a report of de explained variance"""
    pca = PCA(n_components=4)
    pca_result = pca.fit_transform(mat)
    pca_df = pd.DataFrame(columns = ['pca1','pca2','pca3','pca4'])

    pca_df['pca1'] = pca_result[:,0]
    pca_df['pca2'] = pca_result[:,1]
    pca_df['pca3'] = pca_result[:,2]
    pca_df['pca4'] = pca_result[:,3]

    print('Variance explained per principal component: {}'.format(pca.explained_variance_ratio_))

    top_two_comp = pca_df[['pca1','pca2']]

    return top_two_comp.values

def cluster_by_kmeans(mat, n_clusters):
    # Clustering
    data_df = pd.DataFrame(mat)
    k_means = KMeans(random_state=RANDOM_SEED, n_clusters=n_clusters)
    k_means.fit(data_df)
    clusters = np.array(k_means.predict(data_df))
    return clusters

In [ ]:
def add_colored_scatter_plot_to_fig(x, y, colors, rows, columns, n_sub_fig, fig, title):
    ax = fig.add_subplot(rows, columns, n_sub_fig)
    ax.set_title(title)
    sns.scatterplot(x=x, y = y, c=colors, s=80, ax=ax)

def clusters_to_colors(clusters):
    # choose a color palette with seaborn.
    num_classes = len(np.unique(clusters))
    palette = np.array(sns.color_palette("hls", num_classes))
    return palette[clusters.astype(np.int)]

labels = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
               'BMI', 'DiabetesPedigreeFunction', 'Age']
bar_colors = ["blue", "pink", "red", "yellow", "green", "brown", "teal", "goldenrod"]

def visualize_some_explanations_by_cluster(method_class, exp, clusters):
    """ for each method -restricted to one prediction class-
    and for each cluster in that method-prediction-class
    a sample of explanations are presented """
    colors = clusters_to_colors(clusters)

    for i in range(len(np.unique(clusters))):

        # get explanations in cluster of interest
        idx_interest = np.where(clusters==i)[0]
        explanations_in_cluster = exp[idx_interest,:]
        colors_of_interest = colors[idx_interest]

        # select number of explanations to visualize
        n_exp_clus = explanations_in_cluster.shape[0]
        n_explanations = min(n_exp_clus, 10)

        # select sample of explanations to visualize
        idx_interest = np.random.choice(n_exp_clus, n_explanations, replace=False)
        explanations_to_visualize = explanations_in_cluster[idx_interest,:]
        colors_to_visualize = colors_of_interest[idx_interest]

        # Visualization
        rows = n_explanations/2 + 1
        columns = 2
        fig  = plt.figure(figsize = (20, 20))
        title_obj = plt.title(method_class+": examples of explanations in cluster "+str(i))
        plt.setp(title_obj, color=colors_to_visualize[0])

        for j in range(n_explanations):
            ax1 = fig.add_subplot(rows, columns, j+1)
            ax1.set_xlabel("Input features")
            ax1.set_ylabel("Attributions")
            plt.setp(ax1.get_xticklabels(), rotation=15, ha="right",
                 rotation_mode="anchor", fontsize=10)
            barlist = plt.bar(labels, explanations_to_visualize[j].reshape(-1,), color ='black', width = 0.4)
            for k in range(len(labels)):
                barlist[k].set_color(bar_colors[k])
    plt.show()


In [ ]:
for method, (filename, mat) in method_attribution_dict.items():

    rows = 1
    cols = 2
    fig  = plt.figure(figsize = (30, 10))

    proy = get_pca_proj(mat)

    if method[-1] == str(0): clusters = real_y_of_0_preds
    else:
        clusters = real_y_of_1_preds

    colors = clusters_to_colors(clusters)


    add_colored_scatter_plot_to_fig(proy[:,0], proy[:,1], colors, rows, columns, 1, fig,
                                    " PCA true labels")
    # Clustering
    clusters = cluster_by_kmeans(mat, method_elbow_dict[method])

    # Clustering visualization
    colors = clusters_to_colors(clusters)
    add_colored_scatter_plot_to_fig(proy[:,0], proy[:,1], colors, rows, columns, 2, fig,
                                    method+" PCA K-means")

    visualize_some_explanations_by_cluster(method, mat, clusters)



#### SpRAy with Spectral Clustering (SC)

In [ ]:
def generate_graph_laplacian(df, nn):
    """Generate graph Laplacian from data."""
    # Adjacency Matrix.
    connectivity = kneighbors_graph(X=df, n_neighbors=nn, mode='connectivity')
    adjacency_matrix_s = (1/2)*(connectivity + connectivity.T)
    # Graph Laplacian.
    graph_laplacian_s = sparse.csgraph.laplacian(csgraph=adjacency_matrix_s, normed=False)
    graph_laplacian = graph_laplacian_s.toarray()
    return graph_laplacian

def compute_spectrum_graph_laplacian(graph_laplacian):
    """Compute eigenvalues and eigenvectors and project
    them onto the real numbers.
    """
    eigenvals, eigenvcts = linalg.eig(graph_laplacian)

    # Verify eigenvalues are real
    print("Imaginary values ", np.unique(np.imag(eigenvals)))

    eigenvals = np.real(eigenvals)
    eigenvcts = np.real(eigenvcts)
    return eigenvals, eigenvcts

def project_and_transpose(eigenvals, eigenvcts, num_ev):
    """Select the eigenvectors corresponding to the first
    (sorted) num_ev eigenvalues as columns in a data frame.
    """
    eigenvals_sorted_indices = np.argsort(eigenvals)
    indices = eigenvals_sorted_indices[: num_ev]

    proj_df = pd.DataFrame(eigenvcts[:, indices.squeeze()])
    proj_df.columns = ['v_' + str(c) for c in proj_df.columns]
    return proj_df

def run_k_means(df, n_clusters):
    """K-means clustering."""
    k_means = KMeans(random_state=25, n_clusters=n_clusters)
    k_means.fit(df)
    cluster = k_means.predict(df)
    return cluster


def clusters_by_spectral_clustering(mat, n_neighbors, n_clusters):
    """Spectral Clustering Algorithm."""
    df = pd.DataFrame(mat)
    graph_laplacian = generate_graph_laplacian(df, n_neighbors)
    eigenvals, eigenvcts = compute_spectrum_graph_laplacian(graph_laplacian)
    proj_df = project_and_transpose(eigenvals, eigenvcts, n_clusters)
    cluster = run_k_means(proj_df, proj_df.columns.size)

    return np.array(cluster)

First, we decide perplexity for T-SNE in each set of explanations (associated to LRP, LIME or IG and one predicted class),
and the number of clusters for Spectral Clustering.


In [ ]:

for method, (filename, mat) in method_attribution_dict.items():

    perplexitys = [20, 30, 40, 50, 60, 70]
    rows = 2
    columns = (len(perplexitys)+1)/2
    fig  = plt.figure(figsize = (20, 10))

    if method[-1] == str(0):
        clusters = real_y_of_0_preds
    else:
        clusters = real_y_of_1_preds

    colors = clusters_to_colors(clusters)
    plt.title("T-SNE on classified as "+str(method)+" colored by True label ", y=1.2)
    for k, perplexity in enumerate(perplexitys):
        proy = TSNE(random_state=RANDOM_SEED, perplexity=perplexity).fit_transform(mat)
        add_colored_scatter_plot_to_fig(proy[:,0], proy[:,1], colors, rows, columns, k+1, fig,
                                    " perplexity = "+str(perplexity))



In [ ]:
method_class_perplexitys_clusters_dict = dict()

method_class_perplexitys_clusters_dict['lrp_0'] = (30, 2)
method_class_perplexitys_clusters_dict['lrp_1'] = (30, 2)

method_class_perplexitys_clusters_dict['ig_1'] = (20, 3)
method_class_perplexitys_clusters_dict['ig_0'] = (30, 2)

method_class_perplexitys_clusters_dict['lime_0'] = (50, 2)
method_class_perplexitys_clusters_dict['lime_1'] = (60, 2)


method_proy_attribution_labels = dict()
for method, (filename, mat) in method_attribution_dict.items():

    perplexity = method_class_perplexitys_clusters_dict[method][0]

    proy = TSNE(random_state=RANDOM_SEED, perplexity=perplexity).fit_transform(mat)

    if method[-1] == str(0): real_labels = real_y_of_0_preds
    else:
        real_labels = real_y_of_1_preds

    method_proy_attribution_labels[method] = (proy, mat, real_labels)

Election of number of neighbours for Laplacian graph.

In [ ]:
for method, (proy, mat, real_labels) in method_proy_attribution_labels.items():

    neighbors_opt = [5, 10, 20, 30, 40, 50]
    rows = 2
    columns = (len(neighbors_opt)+1)/2
    fig  = plt.figure(figsize = (20, 10))
    plt.title("T-SNE on "+str(method)+" explanations colored by SC cluster ", y=1.2)

    # Clustering
    n_clusters = method_class_perplexitys_clusters_dict[method][1]

    # Clustering visualization
    for k, n_neighbors in enumerate(neighbors_opt):
        clusters = clusters_by_spectral_clustering(mat, n_neighbors, n_clusters)
        colors = clusters_to_colors(clusters)
        add_colored_scatter_plot_to_fig(proy[:,0], proy[:,1], colors, rows, columns, k+1, fig,
                                        "number of neighbors = "+str(n_neighbors))

In [ ]:
method_class_nneighbors = dict()

method_class_nneighbors['lrp_0'] = 30
method_class_nneighbors['lrp_1'] = 30

method_class_nneighbors['ig_1'] = 30
method_class_nneighbors['ig_0'] = 30

method_class_nneighbors['lime_0'] = 10
method_class_nneighbors['lime_1'] = 5


### Visualization of explanations (t-SNE + SC)

All the scatterplots show T-SNE projections of explanations onto firts two principal components.
The explanations in a scatterplot correspond only to one method and to one predicted class, e.g.
"ig_1" is the shorthand for explanations of Integrated gradient (IG), classified as members of class 1
by the model.

Scatterplots at left are colored by their true class, and the ones at right are colored by Spectral clustering (SC)
cluster.

Barplots below are examples of explanations grouped by cluster.

In [ ]:
for method, (proy, mat, real_labels) in method_proy_attribution_labels.items():

    rows = 1
    cols = 2
    fig  = plt.figure(figsize = (30, 10))

    colors = clusters_to_colors(real_labels)
    add_colored_scatter_plot_to_fig(proy[:,0], proy[:,1], colors, rows, columns, 1, fig,
                                    "T-SNE on "+str(method)+" explanations colored by real labels")
    # Clustering
    n_clusters = method_class_perplexitys_clusters_dict[method][1]
    n_neighbors = method_class_nneighbors[method]
    clusters = clusters_by_spectral_clustering(mat, n_neighbors, n_clusters)

    # Clustering visualization
    colors = clusters_to_colors(clusters)
    add_colored_scatter_plot_to_fig(proy[:,0], proy[:,1], colors, rows, columns, 2, fig,
                                    method+" explanations colored by SC cluster")

    visualize_some_explanations_by_cluster(method, mat, clusters)

### Assesing faithfulness

It's crucial that the explanations methods we just revised are **faithfull**, i.e:

"The explainer model sufficiently captures the relationship between the complex model predictions and the features in
the local region around a prediction of interest to produce explanations that are faithful to the complex model."

Towards assesing this, we are going to apply XAI methods in a Logistic regression model, which is already
interpretable.


In [ ]:
dataset = loadtxt('pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

means = np.mean(X, axis=0)
stds = np.std(X, axis=0)
X = (X - means)/stds


In [ ]:
# contruct model
model = Sequential()
model.add(Dense(1, input_dim=8, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, batch_size=10)
score = model.evaluate(X, y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# save model and architecture to single file
model.save("model_logistic_regression.h5")
print("Saved model to disk")

In [ ]:
# load model
model = load_model('model_logistic_regression.h5')
model.summary()
score = model.evaluate(X, y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
preds = model.predict(X) > 0.5
preds = preds.astype(int)
real_y_of_0_preds = y[np.where(preds==0)[0]]
real_y_of_1_preds = y[np.where(preds==1)[0]]

Interpreting the Logistic Model

In [ ]:
# Visualization
rows = 1
columns = 1
fig  = plt.figure(figsize = (10, 10))
title_obj = plt.title("Weight of features in cluster Logistic Regression")
w = model.layers[0].get_weights()
coeff = np.array([item.sum() for sublist in w for item in sublist][:-1])

ax1 = fig.add_subplot(rows, columns, 1)
ax1.set_xlabel("Input features")
ax1.set_ylabel("Attributions")
plt.setp(ax1.get_xticklabels(), rotation=15, ha="right",
     rotation_mode="anchor", fontsize=10)
barlist = plt.bar(labels, coeff.reshape(-1,), color ='black', width = 0.4)
for k in range(len(labels)):
    barlist[k].set_color(bar_colors[k])
plt.show()

In [ ]:
data = []
index = []


n_1_class = np.count_nonzero(preds)
n_0_class = len(preds)-n_1_class
n_explanations = X.shape[0]

eX_1_lrp = np.zeros((n_1_class,8))
eX_0_lrp = np.zeros((n_0_class,8))
eX_1_ig = np.zeros((n_1_class,8))
eX_0_ig = np.zeros((n_0_class,8))


method_attribution_dict = {'ig_1': ("lr_ig_attributtions_1.txt", eX_1_ig),
                                      'lrp_1': ('lr_lrp_attributions_1.txt', eX_1_lrp),
                                      'ig_0': ("lr_ig_attributtions_0.txt", eX_0_ig),
                                      'lrp_0': ('lr_lrp_attributions_0.txt', eX_0_lrp)}

In [ ]:
# Statistics
times_lrp = np.ndarray((n_explanations,))
times_ig = np.ndarray((n_explanations,))

# Counters of explanations per class
j_0 = 0
j_1 = 0

for i in range(n_explanations):

    # input to explain
    xs = X[[i]]

    # For dividing explanations for SpRAy
    x_class = preds[i]

    # Explanations
    with DeepExplain(session=K.get_session()) as de:

        # Need to reconstruct the graph in DeepExplain context, using the same weights.
        # With Keras this is very easy:
        # 1. Get the input tensor to the original model
        input_tensor = model.layers[0].input

        # 2. We now target the output of the last dense layer (pre-softmax)
        # To do so, create a new model sharing the same layers untill the last dense (index -2)
        fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
        target_tensor = fModel(input_tensor)

        # method 1

        # parameters
        arg = 100
        method_name = 'IG'

        # explanations
        begin = time.time()
        explanation = de.explain('intgrad', target_tensor, input_tensor, xs, steps=arg)
        time.sleep(1)
        end = time.time()

        # records
        times_ig[i] = end - begin

        if x_class == 0:
            eX_0_ig[j_0] = explanation
        else:
            eX_1_ig[j_1] = explanation


        # method 2

        # parameters
        arg = 0.0001
        method_name = 'LRP'

        # explanation
        begin = time.time()
        explanation = de.explain('elrp', target_tensor, input_tensor, xs, epsilon=arg)
        time.sleep(1)
        end = time.time()

        # record
        if x_class == 0:
            eX_0_lrp[j_0] = explanation
            j_0 += 1
        else:
            eX_1_lrp[j_1] = explanation
            j_1 += 1

        times_lrp[i] = end - begin

Save explanations and report running time

In [ ]:
method_name = 'IG'
method_dict = {"Mean time": times_ig.mean()}
data.append(method_dict)
index.append(method_name)

method_name = 'LRP'
method_dict = {"Mean time": times_lrp.mean()}
data.append(method_dict)
index.append(method_name)

time_table = pd.DataFrame(data, index=index)
print(time_table)

for (filename, mat) in method_attribution_dict.values():
    mat = np.matrix(mat)
    with open(filename,'wb') as f:
        for line in mat:
            np.savetxt(f, line, fmt='%.2f')

### SpRAy

In [ ]:

for method, (filename, mat) in method_attribution_dict.items():

    perplexitys = [10, 20, 30, 40, 50, 60, 80]
    rows = 2
    columns = (len(perplexitys)+1)/2
    fig  = plt.figure(figsize = (20, 10))

    if method[-1] == str(0):
        clusters = real_y_of_0_preds
    else:
        clusters = real_y_of_1_preds

    colors = clusters_to_colors(clusters)
    plt.title("T-SNE on classified as "+str(method)+" colored by True label ", y=1.2)
    for k, perplexity in enumerate(perplexitys):
        proy = TSNE(random_state=RANDOM_SEED, perplexity=perplexity).fit_transform(mat)
        add_colored_scatter_plot_to_fig(proy[:,0], proy[:,1], colors, rows, columns, k+1, fig,
                                    " perplexity = "+str(perplexity))



In [ ]:
method_class_perplexitys_clusters_dict = dict()

method_class_perplexitys_clusters_dict['lrp_0'] = (80, 2)
method_class_perplexitys_clusters_dict['lrp_1'] = (90, 2)

method_class_perplexitys_clusters_dict['ig_1'] = (50, 2)
method_class_perplexitys_clusters_dict['ig_0'] = (80, 2)

# method_class_perplexitys_clusters_dict['lime_0'] = (50, 2)
# method_class_perplexitys_clusters_dict['lime_1'] = (60, 2)


method_proy_attribution_labels = dict()
for method, (filename, mat) in method_attribution_dict.items():

    perplexity = method_class_perplexitys_clusters_dict[method][0]

    proy = TSNE(random_state=RANDOM_SEED, perplexity=perplexity).fit_transform(mat)

    if method[-1] == str(0): real_labels = real_y_of_0_preds
    else:
        real_labels = real_y_of_1_preds

    method_proy_attribution_labels[method] = (proy, mat, real_labels)

Election of number of neighbours for Laplacian graph.

In [ ]:
for method, (proy, mat, real_labels) in method_proy_attribution_labels.items():

    neighbors_opt = [5, 10, 20, 30, 40, 50]
    rows = 2
    columns = (len(neighbors_opt)+1)/2
    fig  = plt.figure(figsize = (20, 10))
    plt.title("T-SNE on "+str(method)+" explanations colored by SC cluster ", y=1.2)

    # Clustering
    n_clusters = method_class_perplexitys_clusters_dict[method][1]

    # Clustering visualization
    for k, n_neighbors in enumerate(neighbors_opt):
        clusters = clusters_by_spectral_clustering(mat, n_neighbors, n_clusters)
        colors = clusters_to_colors(clusters)
        add_colored_scatter_plot_to_fig(proy[:,0], proy[:,1], colors, rows, columns, k+1, fig,
                                        "number of neighbors = "+str(n_neighbors))

In [ ]:
method_class_nneighbors = dict()

method_class_nneighbors['lrp_0'] = 30
method_class_nneighbors['lrp_1'] = 10

method_class_nneighbors['ig_1'] = 20
method_class_nneighbors['ig_0'] = 10

# method_class_nneighbors['lime_0'] = 10
# method_class_nneighbors['lime_1'] = 5


### Visualization of explanations (t-SNE + SC)

All the scatterplots show T-SNE projections of explanations onto firts two principal components.
The explanations in a scatterplot correspond only to one method and to one predicted class, e.g.
"ig_1" is the shorthand for explanations of Integrated gradient (IG), classified as members of class 1
by the model.

Scatterplots at left are colored by their true class, and the ones at right are colored by Spectral clustering (SC)
cluster.

Barplots below are examples of explanations grouped by cluster.

In [ ]:
for method, (proy, mat, real_labels) in method_proy_attribution_labels.items():

    rows = 1
    cols = 2
    fig  = plt.figure(figsize = (30, 10))

    colors = clusters_to_colors(real_labels)
    add_colored_scatter_plot_to_fig(proy[:,0], proy[:,1], colors, rows, columns, 1, fig,
                                    "T-SNE on "+str(method)+" explanations colored by real labels")
    # Clustering
    n_clusters = method_class_perplexitys_clusters_dict[method][1]
    n_neighbors = method_class_nneighbors[method]
    clusters = clusters_by_spectral_clustering(mat, n_neighbors, n_clusters)

    # Clustering visualization
    colors = clusters_to_colors(clusters)
    add_colored_scatter_plot_to_fig(proy[:,0], proy[:,1], colors, rows, columns, 2, fig,
                                    method+" explanations colored by SC cluster")

    visualize_some_explanations_by_cluster(method, mat, clusters)

